In [1]:
import numpy as np
import pandas as pd
import random as rd
import datetime
import os
import itertools

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error

In [ ]:
# Define some constants that we will use
PROJECT_PATH = '/home/ubuntu/projects/kaggle-predict-future-sales'

TRAIN = 'train'
VALID = 'valid'
TEST = 'test'

In [ ]:
# Read in raw CSV files

csv_data = dict()

# Import all csv file data
csv_data[ 'sales_daily' ] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/sales_train.csv') )
csv_data['item_cat'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/item_categories.csv') )
csv_data['item'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/items.csv') )
csv_data['sub'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/sample_submission.csv') )
csv_data['shops'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/shops.csv') )
csv_data['test'] = pd.read_csv( os.path.join( PROJECT_PATH, 'input/test.csv') )

In [ ]:
# Add some new columns and aggregate from daily to monthly
############################################################

# Add the daily revenue
csv_data['sales_daily'][ "revenue" ] = csv_data['sales_daily'].item_price * csv_data['sales_daily'].item_cnt_day

# Aggregate the monthly data
agg_rules = {'item_price' : "mean", "revenue" : "sum", "item_cnt_day" : "sum" }
groupby_cols = [ 'date_block_num', "item_id", "shop_id" ]

# Add the effective item price
csv_data['sales_monthly'] = csv_data['sales_daily'].groupby( groupby_cols ).agg( agg_rules ).reset_index()

# Rename the column to reflect monthly data
csv_data['sales_monthly'].rename( columns={ 'item_cnt_day' : 'item_cnt_month'}, inplace=True )

# Add unit price as a column
csv_data['sales_monthly']['item_price_unit'] = np.round( csv_data['sales_monthly']['revenue'] / \
                              ( 1e-6 + csv_data['sales_monthly']['item_cnt_month'] ) )


In [ ]:
# Get all combinations of shops/items for each date
############################################################

df_monthly = csv_data['sales_monthly']
dates = df_monthly['date_block_num'].unique()

df_id = pd.DataFrame( [], columns=['date_block_num', 'shop_id', 'item_id'])
for dt in dates:
        
    df_t = df_monthly[ df_monthly['date_block_num'] == dt ]
    uniq_shops = df_t['shop_id'].unique()    
    uniq_items = df_t['item_id'].unique()
        
    new_rows = pd.DataFrame( itertools.product( [dt], uniq_shops, uniq_items ), columns=df_id.columns )
    df_id = pd.concat( [ df_id, new_rows ], sort=False, axis=0 )

# Join the test IDs to the data frame
df_t = csv_data['test'].copy()
df_t['date_block_num'] = 1 + df_id['date_block_num'].max()
df_id = pd.concat( [ df_id, df_t ], sort=False, axis=0 ).drop(['ID'], axis=1)

# Create a data frame using all the shop/item pairs from each month
df_sales = df_id.merge( csv_data['sales_monthly'], on=['date_block_num', 'shop_id', 'item_id' ], how='left' )

# Set missing values for revenue and item count to 0
df_sales.loc[ np.isnan(df_sales['revenue']), ['revenue'] ] = 0
df_sales.loc[ np.isnan(df_sales['item_cnt_month']), ['item_cnt_month'] ] = 0

In [ ]:
# Add new columns to the data frame
############################################################

# Add the category id
df_sales = df_sales.merge( csv_data['item'], on='item_id' )
df_sales = df_sales.drop('item_name', axis=1 )

# Add a column combining shop and item, which together with date_block_num is a unique id
df_sales['shop_item_id'] = df_sales['shop_id'] + 100 * df_sales['item_id']

# Add the month and year
df_sales['month'] = 1 + (df_sales['date_block_num'] % 12)
df_sales['year'] = 2013 + (df_sales['date_block_num'] // 12)

# Add the dates
# dates = pd.Series( [ datetime.date( y, m, 15 ) for y in range(2013,2016) for m in range(1, 13) ], \
#                           index=range(0,36) )
# df_sales['date'] = df_sales['date_block_num'].map(dates)

# Rename the target column and make it the first column
target = df_sales.loc[:,['item_cnt_month']]
target.columns = [ 'TARGET']
df_sales = pd.concat( [ target, df_sales.drop('item_cnt_month', axis=1) ], axis=1 ) 


In [ ]:
%%time
# Make time series out of the monthly  sales
pivot_columns = [ 'shop_id', 'item_id']
target_col = 'item_cnt_month'
grp_table = csv_data['sales_monthly'].groupby( [ 'date_block_num' ] + pivot_columns ).agg( { target_col : 'sum' } ).reset_index()
ts_sales_raw = grp_table.pivot_table( target_col, index="date_block_num", columns=pivot_columns )

# Set negative values to 0
ts_sales_raw = np.maximum( 0, ts_sales_raw )

# Make sure the dates are sorted
ts_sales_raw.sort_index(axis=0, ascending=True, inplace=True )

# Create a version that has missing values filled with 0
ts_sales = ts_sales_raw.fillna(0)

# Downcast both versions
ts_sales_raw = ts_sales_raw.astype('float32')
ts_sales = ts_sales.astype('float32')

In [ ]:
%%time
# Initialize a list to store features. We will concatenate these afterwards with hstack
feature_vals = []
feature_names = []

# Just keep the last M months of observations, and convert to a numpy column vector
process_features = lambda x : x.to_numpy().ravel()[:,np.newaxis]

# Get lagged monthly sales
print( 'Calculating lagged sales...')
lags = np.arange(1,13)
for L in lags:
    ts = ts_sales.shift(periods=L-1)
    feature_vals.append( process_features(ts) )
    feature_names.append( 'sales_lag_{:02}'.format(L) )

# Get lagged monthly mean sales
print( 'Calculating rolling mean sales...')
means = [3, 6, 9, 12]
for M in means:
    ts = ts_sales.rolling(window=M).mean()
    feature_vals.append( process_features(ts) )    
    feature_names.append( 'sales_mean_{:02}'.format(M) )

# Get 12-month standard deviation
print( 'Calculating std. dev. of sales...')
means = [6, 12]
for M in means:
    ts = ts_sales.rolling(window=M).std()
    feature_vals.append( process_features(ts) )
    feature_names.append( 'sales_std_{:02}'.format(M) )

# Get the 12-month quartiles
print( 'Calculating quantiles of sales...')
for p in [ 0.25, 0.50, 0.75]:
    ts = ts_sales.rolling(window=12).quantile(p)
    feature_vals.append( process_features(ts) )    
    feature_names.append( 'sales_percentile_{:02}_12'.format(int(p * 100) ) )

# Add some categorical features
print( 'Collecting categorical features...')
for j, col in enumerate(ts_sales.columns.names):
    ts = np.array([ x[j] for x in ts_sales.columns ] * ts_sales.shape[0]).reshape( ts_sales.shape[0], ts_sales.shape[1])
    feature_vals.append( process_features(pd.DataFrame(ts) ) )  
    feature_names.append( col )

# Add some more categorical features for the date block, year and month
ts_dates = np.tile( np.array(ts_sales.index)[:,np.newaxis], ts_sales.shape[1] )
feature_vals.append( process_features(pd.DataFrame(ts_dates) ) )
feature_names.append( 'date_block_num' )


In [ ]:
# Downcast floats and ints to 32-bit to save memory
################################################################

def downcast_dataframe( df ):
    for col in df.columns:
        if isinstance( df[col ].iloc[0], np.float64 ) or isinstance( df[col ].iloc[0], float ):
            df[col] = df[col].astype('float32')
        elif isinstance( df[col ].iloc[0], np.int64 ) or isinstance( df[col ].iloc[0], int ):
            df[col] = df[col].astype('int32')
            
    return df


In [ ]:
def get_months_since_first_and_last_observation( input_ts ):
    """Gets the number of months since the first and last observation in each columns, 
    at each point in time. This function has no 'look-forward' bias."""
    
    # Create an array with the index (0 to T) of any non-zeros entries
    month_of_obs = input_ts.to_numpy() * np.arange(1,input_ts.shape[0]+1)[:,np.newaxis]
    month_of_obs = month_of_obs.astype('float32')
    month_of_obs[ month_of_obs < 0.01 ] = np.nan

    # Make a data frame
    df_month_of_obs = pd.DataFrame( month_of_obs, columns=input_ts.columns)

    # Loop through the time steps and find the months since first/last action at each t
    months_since_first_obs = np.nan * np.ones_like(month_of_obs)
    months_since_last_obs = np.nan * np.ones_like(month_of_obs)
    for t in range( month_of_obs.shape[0] ):
        months_since_first_obs[t,:] = t - df_month_of_obs.iloc[:(t+1),:].idxmin(axis=0)
        months_since_last_obs[t,:] = t - df_month_of_obs.iloc[:(t+1),:].idxmax(axis=0)    

    # Fill NaN's with 999 for shop/items that have never seen a sale
    months_since_first_obs[ np.isnan( months_since_first_obs ) ] = 999
    months_since_last_obs[ np.isnan( months_since_last_obs ) ] = 999

    # Convert back into pandas Dataframe
    months_since_first_obs = pd.DataFrame( months_since_first_obs, columns=ts_sales.columns )
    months_since_last_obs = pd.DataFrame( months_since_last_obs, columns=ts_sales.columns )
    
    return months_since_first_obs, months_since_last_obs


In [ ]:
%%time
# Get the number of months since the first and last sale, at each point in time
months_since_first_sale, months_since_last_sale = \
        get_months_since_first_and_last_observation( ts_sales_raw )

# Add to features
feature_vals.append( process_features( months_since_first_sale.astype('float32') ) )
feature_names.append( 'months_since_first_sale' )
feature_vals.append( process_features( months_since_last_sale.astype('float32') ) )
feature_names.append( 'months_since_last_sale' )

In [ ]:
%%time 
# Join the features into a pandas data frame
features_list = [ x for x in feature_vals ]
df_calc = pd.DataFrame( np.hstack(features_list), columns=feature_names)

# We must shift the date_block_num forward by 1 for the calculated columns, 
#    since these are not available until the next month
df_calc['date_block_num'] = 1 + df_calc['date_block_num'].values


In [ ]:
%%time
# Combine the sales data with the calculated features
df_full = df_sales.merge( df_calc, on=['date_block_num', 'shop_id', 'item_id' ], how='left' )
df_full = downcast_dataframe(df_full)

# Remove the first 12 months, since features using 12 months of data will be NaN
df_full = df_full[ df_full['date_block_num'] >= 12 ]

# Clean up missing values in the calculated columns
calc_sales_cols = list( df_calc.columns[ [ x.startswith('sales_') for x in df_calc.columns ] ] )
df_full[calc_sasles_cols] = df_full[calc_sales_cols].fillna(0)

df_full['months_since_first_sale'] = df_full['months_since_first_sale'].fillna(999).values
df_full['months_since_last_sale'] = df_full['months_since_last_sale'].fillna(999).values


In [ ]:
# Add additional grouped features 
#############################################

def create_group_features( df, group_col, agg_rule, target_col ):
    new_col_name = target_col + '_' + agg_rule + '_by_' + group_col 
    vals = df_full.groupby( [ 'date_block_num', group_col ] )[ target_col ].transform(agg_rule)
    df[new_col_name] = vals.astype('float32')    
    return df

# Create a new column representing the total sales over the past 12 months
df_full['sales_total_12'] = 12 * df_full['sales_mean_12']

for target_col in [ 'sales_lag_01', 'sales_total_12' ]:
    for agg_rule in [ 'sum', 'mean', 'count' ]:
        for group_col in [ 'item_id', 'shop_id', 'item_category_id' ]:
            df_full = create_group_features( df_full, group_col, agg_rule, target_col )

for group_col in [ 'item_id', 'shop_id', 'item_category_id']:
    df_full = create_group_features( df_full, group_col, 'min', 'months_since_first_sale' )
    df_full = create_group_features( df_full, group_col, 'min', 'months_since_last_sale' )   
    

In [ ]:
# Add additional categorical features
#############################################

df_full['shop_item_never_active'] = ( df_full['months_since_first_sale'] == 999 ).astype('int32')
df_full['shop_item_inactive'] = ( df_full['months_since_last_sale'] > 12 ).astype('int32')


In [ ]:
# Define the base set of features that will be used for training and validation
########################################################################

# Drop the revenue and price columns
df_base = df_full.drop( [ 'item_price', 'item_price_unit', 'revenue' ], axis=1 )

# Only keep the past 12 periods for testing
df_base = df_base[ df_base['date_block_num' ] >= 22 ]
df_base = downcast_dataframe( df_base )


In [ ]:
# Save the raw features to a HDF5 file
print( 'File size: {:02} MB'.format( np.round( df_base.memory_usage().sum() / 1e6 ) ) )
# file_name = os.path.join( PROJECT_PATH, 'preprocessed_data/base_features.h5')
# df_base.to_hdf(file_name, key='base_features')     


In [ ]:
# Read in the raw features from a HDF5 file
# file_name = os.path.join( PROJECT_PATH, 'preprocessed_data/raw_features.h5')
# df_full = pd.read_hdf(file_name, key='raw_features')

In [ ]:
sales_cols = df_base.columns[ [ x.startswith( 'sales_lag_') or x.startswith( 'sales_mean_') for x in df_base.columns ] ]
X = df_base[ sales_cols ]

from sklearn.decomposition import PCA

pc = PCA( n_components=10 )
pc.fit(X)


In [ ]:
pc.explained_variance_ratio_

In [ ]:
# Define functions for mean encoding
########################################################################

def encode_means_with_cv( df, target_col, group_col, n_splits ):

    kf = KFold(n_splits=n_splits, shuffle=False)
    split_info = [ x for x in kf.split(df) ]

    mu = np.nanmean( df[target_col] )
    encoded_feature = pd.Series( np.nan * np.ones_like(df[target_col]), index = df[group_col] )

    for splt in split_info:
        # Get the test and train indices for the current fold
        idx_train, idx_test = splt

        # Get the test and train data
        train_data = df.iloc[idx_train,:]
        test_data = df.iloc[idx_test,:]

        # Put the means into the output vector
        encoded_feature.iloc[idx_test] = encode_means_from_test_train_split( train_data, test_data, target_col, group_col)

    # Fill missing values with the global mean
    encoded_feature = encoded_feature.fillna(mu)
    
    return encoded_feature


def encode_means_from_test_train_split( train_data, test_data, target_col, group_col):
    
    # Get item IDs common to both test and train, and also those just found in the test set
    common_ids = set(test_data[group_col]).intersection( set(train_data[group_col]) )
    missing_ids = set(test_data[group_col]).difference(common_ids)

    # Construct a dictionary mapping item IDs from the test set to their means in the train set
    train_means = train_data.groupby(group_col)[target_col].mean()    
    common_means = pd.Series( [ train_means[x] for x in list(common_ids) ], index=pd.Index(common_ids, dtype='int64' ) )
    missing_means = pd.Series( np.nan * np.ones_like(missing_ids), index=pd.Index(list(missing_ids), dtype='int32' ) )
    all_means = dict(pd.concat( [ common_means, missing_means ] ) )

    encoded_features_test = [ all_means[x] for x in test_data[group_col] ]    
    return encoded_features_test    


In [ ]:
# Get different datasets, in chronological order
# The test set is always the next period after the end of the train set.
##########################################################################

def get_validation_set( df, idx, min_train_size=10 ):

    date_blocks = df['date_block_num']
    uniq_date_blocks = np.sort( date_blocks.unique() )

    n_datasets = len(uniq_date_blocks) - min_train_size - 1
    if idx >= 0:
        split_date = uniq_date_blocks[min_train_size + idx]
    else:
        split_date = uniq_date_blocks[idx]    

    xtrain = df[ date_blocks < split_date].drop('TARGET', axis=1 )
    ytrain = df[ date_blocks < split_date ]['TARGET'][:,np.newaxis]
    xtest = df[ date_blocks == split_date ].drop('TARGET', axis=1 )
    ytest = df[ date_blocks == split_date ]['TARGET'][:,np.newaxis]

    return xtrain, ytrain, xtest, ytest


In [ ]:
n_splits = 5
group_cols = [ 'item_id', 'shop_id', 'item_category_id', 'shop_item_id', 'date_block_num', 'month' ]

for group_col in group_cols:

    new_col_name = 'TARGET' + '_mean_' + group_col
    print('{}'.format(datetime.datetime.now()) + ' : ' + new_col_name )

    if new_col_name not in df_dict[TRAIN]:
        df_dict[TRAIN][new_col_name] = encode_means_with_cv( df_dict[TRAIN], \
                                    target_col='TARGET', group_col=group_col, n_splits=n_splits ).to_numpy()

    if new_col_name not in df_dict[VALID]:
        df_dict[VALID][new_col_name] = encode_means_from_test_train_split( df_dict[TRAIN], df_dict[VALID], \
                                    target_col='TARGET', group_col=group_col)    

    if new_col_name not in df_dict[TEST]:
        test_data = pd.concat( [ df_dict[TRAIN], df_dict[VALID] ])                
        df_dict[TEST][new_col_name] = encode_means_from_test_train_split( test_data, df_dict[VALID], \
                                    target_col='TARGET', group_col=group_col)    

In [ ]:
# Construct some train/validation/test data sets
##########################################################################

data_sets = []
for j in range(100):
    try:
        xtrain, ytrain, xtest, ytest = get_validation_set( df_full, j )
        data_sets.append( ( xtrain, ytrain, xtest, ytest ) )
    except IndexError:
        break